# Hyperparameter Tuning using HyperDrive

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Environment, ScriptRunConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import pandas as pd
import os
import joblib

In [2]:
ws = Workspace.from_config()
ws.write_config(path='.azureml')
experiment_name = 'capstone-gridstability-hd'

exp = Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

print(ws)

Workspace name: quick-starts-ws-183770
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-183770
Workspace.create(name='quick-starts-ws-183770', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-183770')


In [3]:
cpu_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Cluster already exists. Use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)

Cluster already exists. Use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
env = Environment.get(workspace = ws, name = "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")

In [5]:
if "training" not in os.listdir():
    os.mkdir("./training")

In [6]:
src = ScriptRunConfig(source_directory = ".",
                      script = "train.py",
                      compute_target = "my-cluster",
                      environment = env
                     )

## Dataset

In [7]:
os.listdir()

['.amlignore',
 '.amlignore.amltmp',
 '.azureml',
 '.ipynb_aml_checkpoints',
 'automl.ipynb',
 'best_model_hd.sav',
 'data',
 'hyperparameter_tuning.ipynb',
 'hyperparameter_tuning.ipynb.amltmp',
 'outputs',
 'train.py',
 'train.py.amltmp',
 'training']

In [8]:
ds = pd.read_csv("data/gridstability.csv")

In [9]:
ds.describe()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5.250000,5.250001,5.250004,5.249997,3.750000,-1.250000,-1.250000,-1.250000,0.525000,0.525000,0.525000,0.525000,0.015731
std,2.742548,2.742549,2.742549,2.742556,0.752160,0.433035,0.433035,0.433035,0.274256,0.274255,0.274255,0.274255,0.036919
min,0.500793,0.500141,0.500788,0.500473,1.582590,-1.999891,-1.999945,-1.999926,0.050009,0.050053,0.050054,0.050028,-0.080760
25%,2.874892,2.875140,2.875522,2.874950,3.218300,-1.624901,-1.625025,-1.624960,0.287521,0.287552,0.287514,0.287494,-0.015557
50%,5.250004,5.249981,5.249979,5.249734,3.751025,-1.249966,-1.249974,-1.250007,0.525009,0.525003,0.525015,0.525002,0.017142
75%,7.624690,7.624893,7.624948,7.624838,4.282420,-0.874977,-0.875043,-0.875065,0.762435,0.762490,0.762440,0.762433,0.044878
max,9.999469,9.999837,9.999450,9.999443,5.864418,-0.500108,-0.500072,-0.500025,0.999937,0.999944,0.999982,0.999930,0.109403


## Hyperdrive Configuration

In [10]:
# Specify parameter sampler
ps = RandomParameterSampling({"--C":choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
                             "--max_iter":choice(50,100,200,300)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=1)

In [11]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hd_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             policy=policy,
                             max_total_runs=16,
                             max_concurrent_runs=4)

## Run Details

In [12]:
# Submit the hyperdrive run to the experiment and show run details with the widget.
hd_run = exp.submit(hd_config)

# The widget RunDetails can be used to monitor the run
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [13]:
hd_run.wait_for_completion(show_output=True)

RunId: HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2
Web View: https://ml.azure.com/runs/HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-183770/workspaces/quick-starts-ws-183770&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-02-11T12:49:02.965697][API][INFO]Experiment created<END>\n"

Execution Summary
RunId: HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2
Web View: https://ml.azure.com/runs/HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-183770/workspaces/quick-starts-ws-183770&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2',
 'target': 'my-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-11T12:49:02.516455Z',
 'endTimeUtc': '2022-02-11T12:57:40.738085Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '46c56042-db29-4aa7-930b-b79b51c2f69e',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1064-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.37.0',
  'space_size': '44',
  'score': '0.817',
  'best_child_run_id': 'HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2_3',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg183770.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2/azureml-logs/hyperdrive

In [14]:
print(hd_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

[{'run_id': 'HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2_6', 'hyperparameters': '{"--C": 500, "--max_iter": 50}', 'best_primary_metric': 0.817, 'status': 'Completed'}, {'run_id': 'HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2_3', 'hyperparameters': '{"--C": 20, "--max_iter": 50}', 'best_primary_metric': 0.817, 'status': 'Completed'}, {'run_id': 'HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2_13', 'hyperparameters': '{"--C": 1, "--max_iter": 300}', 'best_primary_metric': 0.8165, 'status': 'Completed'}, {'run_id': 'HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2_9', 'hyperparameters': '{"--C": 1000, "--max_iter": 50}', 'best_primary_metric': 0.8165, 'status': 'Completed'}, {'run_id': 'HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2_5', 'hyperparameters': '{"--C": 1, "--max_iter": 100}', 'best_primary_metric': 0.8165, 'status': 'Completed'}, {'run_id': 'HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2_14', 'hyperparameters': '{"--C": 100, "--max_iter": 200}', 'best_primary_metric': 0.816, 'status': 'Completed'}, {'run_id': 'HD_b2

## Best Model

In [15]:
print(hd_run.get_children_sorted_by_primary_metric(top=1, reverse=False, discard_no_metric=False))

[{'run_id': 'HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2_6', 'hyperparameters': '{"--C": 500, "--max_iter": 50}', 'best_primary_metric': 0.817, 'status': 'Completed'}]


In [16]:
best_run = hd_run.get_best_run_by_primary_metric()
best_run.get_file_names()

['outputs/model.pkl',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'user_logs/std_log.txt']

In [17]:
best_run.get_metrics()

{'Regularization Strength:': 20.0, 'Max iterations:': 50, 'Accuracy': 0.817}

In [18]:
best_run.get_details()

{'runId': 'HD_b2f3f053-dc2e-42cc-99a6-61575ed6ece2_3',
 'target': 'my-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-11T12:50:37.504425Z',
 'endTimeUtc': '2022-02-11T12:50:41.206911Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '46c56042-db29-4aa7-930b-b79b51c2f69e',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '20', '--max_iter', '50'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'my-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'en

In [23]:
best_run.register_model(model_name = 'best_model_hd', model_path='outputs/model.pkl')

Model(workspace=Workspace.create(name='quick-starts-ws-183770', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-183770'), name=best_model_hd, id=best_model_hd:1, version=1, tags={}, properties={})